In [1]:
import pandas as pd
import numpy as np

In [2]:
authorInt = pd.read_csv("authorToInt.csv")
new_mapping = {}

In [3]:
for ind in authorInt.index:
    a = authorInt["id"][ind]
    b = authorInt["new_id"][ind]
    new_mapping[a] = b

# Community change

In [4]:
commdf = pd.read_csv("community.csv")

In [13]:
onlyInt = authorInt.drop(["author"], axis=1)

In [14]:
newdf = pd.merge(commdf, onlyInt, left_on="author", right_on="id", how ="right")

In [16]:
# newdf = newd

,author,community,id,new_id
0,0.0,0.0,0,6782
1,1.0,0.0,1,6783
2,2.0,0.0,2,6784
3,3.0,1.0,3,2
4,4.0,1.0,4,3


In [ ]:
# authorInt.describe()

In [20]:
# newdf = newdf.drop(["author", "id"], axis=1)
# newdf.rename(columns={"new_id": "id"}, inplace=True)
newdf = newdf.dropna()

In [21]:
newdf.describe()

,community,id
count,7015.000000,7015.000000
mean,76.724733,3546.414113
std,110.877223,2051.756916
min,0.000000,0.000000
25%,14.000000,1767.500000
50%,25.000000,3547.000000
75%,63.000000,5322.500000
max,459.000000,7101.000000


In [23]:
newdf.to_csv("n_community.csv", index=False)

# Convert df to matrix files

In [ ]:
def dfToMat(df):
    columns = df.columns
    mat = []
    for i in columns:
        mat.append(df[i].to_list())
    
    return mat

# Pearson distance

In [ ]:
pearson = pd.read_csv("./layer3/pearson_distance.csv.zip")
# pearson.to_csv("tmp.csv")

In [ ]:
import numpy as np

In [ ]:
def dfToMat(df):
	mat = []
	for i in range(7102):
		mat.append(df[str(i)].to_list())
	return mat

In [ ]:
mat = dfToMat(pearson)

In [ ]:
def pad(n):
    n = str(n)
    return "0"*(4-len(n)) + n

In [ ]:
print(len(mat))

In [ ]:
n_mat = np.zeros((7102, 7102))

In [ ]:
for i in range(len(mat)):
    for j in range(len(mat[i])):
        n_mat[new_mapping[i]][new_mapping[j]] = mat[i][j]
    # if i not in new_mapping:
    #     print(i)

In [ ]:
for i in range(len(mat)):
    # f.open("layer3/pearson/" + str(pad(i)), "w")
    np.array(mat[i]).tofile("layer3/pearson/" + str(i))
    if i % 100 == 0:
        print(i)

# Layer 2

In [ ]:
mat = np.reshape(np.fromfile("layer2/sim_mat", dtype=float), (7102, 7102))

In [ ]:
n_mat = np.zeros((7102, 7102))

In [ ]:
for i in range(len(mat)):
    for j in range(len(mat[i])):
        n_mat[new_mapping[i]][new_mapping[j]] = mat[i][j]
    # if i not in new_mapping:
    #     print(i)

In [ ]:
for i in range(len(mat)):
    # f.open("layer2/similarity/" + str(pad(i)), "w")
    np.array(mat[i]).tofile("layer2/similarity/" + str(i))
    if i % 1000 == 0:
        print(i)

# edge list to new ids

In [24]:
import glob

In [32]:
mapping = pd.read_csv("authorToInt.csv")
mapping = mapping.drop(["author"], axis=1)

In [33]:
mp = {}
for ind in mapping.index:
	mp[mapping["id"][ind]] = mapping["new_id"][ind]

In [34]:
community = glob.glob("community/*")

In [35]:
print(community[1])

community\1.edge


In [37]:
for comm in community:
	s = ""
	f = open(comm, "r")
	for line in f.readlines():
		a, b = map(int, line[:-1].split())
		s += "{} {}\n".format(str(mp[a]), str(mp[b]))
	f.close()
	f = open("edgefiles/{}".format(comm.split("\\")[-1]), "w")
	f.write(s)
	f.close()

# combined df

In [29]:
def getJointMat(path):
	mat = []
	for i in glob.glob(path):
		mat.append(np.fromfile(i, dtype=float))
	
	return mat
		

In [30]:
layer2 = getJointMat("layer2/similarity/*")
layer3 = getJointMat("layer3/pearson/*")

In [39]:
combined = []
edge_files = glob.glob("edgefiles/*")
for fle in edge_files:
    f = open(fle, "r")
    comm = fle.split("\\")[-1].strip(".edge")
    for line in f.readlines():
        a, b = map(int, line[:-1].split())
        combined.append([a, b, comm, layer2[a][b], layer3[a][b]])
        combined.append([b, a, comm, layer2[a][b], layer3[a][b]])
    f.close()

In [40]:
c_df = pd.DataFrame(data=combined, columns=["node1", "node2", "community", "topic_similarity", "temporal_similarity"])

In [42]:
c_df.to_csv("combined.csv", index=False)

In [ ]:
# for file in edge_files:
#     f = open(file, "r")
#     for line in f.readlines():
#         a, b = map(int, line[:-1].split())
#         if a not in new_mapping:
#             new_mapping[a] = count
#             count += 1
#         if b not in new_mapping:
#             new_mapping[b] = count
#             count += 1
#         tmp.append([new_mapping[a], new_mapping[b], df_weight.loc[a].at[str(b)]])
#         tmp.append([new_mapping[b], new_mapping[a], df_weight.loc[a].at[str(b)]])
#     f.close()